### Exploration
- all name V

### column
nama, nik, tempatLahir, jenisKelamin, tps, provinsi, kab_kota, 

kecamatan, desa, newcomer, nama_sunda, nama_jawa, nama_batak, 

bintang, inisial_r, nama_kristen, umur, tahun_lahir, tanggal_produksi, 

hari_produksi, unique_location, unique_hash, lat, long, count, unique_location:1,

 lat:1, long:1, lat_clean, long_clean

In [1]:
import pandas as pd
import plotly.graph_objects as go


In [2]:
df = pd.read_csv('name_composition.csv')
df

,source,target,value
0,AEP,SAEPUL,1485
1,SAEPUL,MALIK,869
2,RUDI,SUNDARI,3
3,ACEP,AAM,8
4,SARAH,YULIA,8
...,...,...,...
7408748,IDA,MUNUMG,1
7408749,WILSA,NOVA,1
7408750,DEAN,RAMADAN,1
7408751,SUKATMA,ISMAIL,1


In [10]:
name_nodes_df_filtered = df.sort_values('value',ascending=False).head(20)
name_nodes_df_filtered

,source,target,value
1051,SITI,AMINAH,22503
13,SITI,AISAH,22474
768,SRI,RAHAYU,21051
1469,ABDUL,ROHMAN,18251
389,SITI,AISYAH,18048
6,SITI,NURJANAH,17549
4932,SRI,WAHYUNI,16400
2172,SITI,FATIMAH,15591
1305,IWAN,SETIAWAN,15022
319,RAHMAT,HIDAYAT,14219


In [11]:
import plotly.graph_objects as go
import pandas as pd

# Sample DataFrame
sample_df = pd.DataFrame({
    'source': ['Node A', 'Node A', 'Node B', 'Node C', 'Node C'],
    'target': ['Node B', 'Node C', 'Node D', 'Node E', 'Node F'],
    'value': [10, 5, 15, 8, 12]
})

# Define nodes and links
nodes = pd.unique(name_nodes_df_filtered[['source', 'target']].values.ravel('K'))
links = name_nodes_df_filtered[['source', 'target', 'value']]

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color='black', width=0.5),
        label=nodes
    ),
    link=dict(
        source=links['source'].map(lambda x: nodes.tolist().index(x)),
        target=links['target'].map(lambda x: nodes.tolist().index(x)),
        value=links['value']
    )
)])

# Customize the appearance of the diagram
fig.update_layout(title_text='Sankey Diagram', font=dict(size=12))

# Show the plot
fig.show()


In [3]:
names = pd.read_csv('names.csv')
names.head(3)

,nama,unique,1stname,2ndname,3rdname,4thname,5thname,lastname1,lastname2,lastname3,lastname4,lastname5
0,NYANYAH,NaN,NYANYAH,NaN,NaN,NaN,NaN,NYANYAH,NaN,NaN,NaN,NaN
1,JALALUDIN,NaN,JALALUDIN,NaN,NaN,NaN,NaN,JALALUDIN,NaN,NaN,NaN,NaN
2,AEP SAEPUL MALIK,AEP,AEP,SAEPUL,MALIK,NaN,NaN,MALIK,SAEPUL,AEP,NaN,NaN


In [21]:
name_nodes = {}
def create_node(row,key1,key2):
    return dict(
                source=row[key1],
                target=row[key2],
                value=1
            )
def name_nodes_append(name_nodes,row,key1,key2):
    if type(row[key1]) == str and type(row[key2]) == str:
        key = row[key1]+'_'+row[key2]
        if key not in name_nodes:
            name_nodes[key] = create_node(row,key1,key2)
        else:
            name_nodes[key]['value'] += 1

    return name_nodes

for id,row in names[['lastname1','lastname2','lastname3','lastname4','lastname5']].iterrows():
    name_nodes = name_nodes_append(name_nodes,row,'lastname1','lastname2')
    name_nodes = name_nodes_append(name_nodes,row,'lastname2','lastname3')
    name_nodes = name_nodes_append(name_nodes,row,'lastname3','lastname4')
    name_nodes = name_nodes_append(name_nodes,row,'lastname4','lastname5')

    
name_nodes_df = pd.DataFrame([name_nodes[key] for key in name_nodes])
name_nodes_df

,source,target,value
0,MALIK,SAEPUL,484
1,SAEPUL,AEP,817
2,SUNDARI,RUDI,2
3,AAM,ACEP,4
4,YULIA,SARAH,6
...,...,...,...
5289903,FACHRIZAL,RATNO,1
5289904,RAHARDJO,TJAHYONO,1
5289905,AHADIN,FAJAR,1
5289906,SHOLIHAH,THOATIN,1


In [22]:
name_nodes_df.to_csv('name_composition2.csv',index=False)

In [2]:
name_nodes_df = pd.read_csv('name_composition2.csv')
name_nodes_df.head(2)

,source,target,value
0,MALIK,SAEPUL,484
1,SAEPUL,AEP,817


In [25]:
names['lastname1'].value_counts(ascending=False).head(15)

lastname1
HIDAYAT      107315
SETIAWAN      88841
NURHAYATI     86235
SARI          84426
GUNAWAN       61690
MAULANA       54404
SUPRIATNA     52573
S             50883
SUMIATI       50428
MULYANA       50407
SAEPUDIN      48975
LESTARI       48961
SURYANA       48239
SAPUTRA       48023
HERMAWAN      48017
Name: count, dtype: int64

In [90]:
from tqdm import tqdm
def find_name_leaf(lastname1=None,lastname2=None,lastname3=None,lastname4=None,size=5):
    names_chunk = pd.DataFrame()
    filter_name = 'lastname1'
    if lastname1 == None:
        names_chunk = names.copy()
    if lastname1:
        names_chunk = names[names['lastname1']==lastname1[0]]
        filter_name = 'lastname2'
    if lastname2:
        names_chunk = names_chunk[names_chunk['lastname2']==lastname2[0]]
        filter_name = 'lastname3'
    if lastname3:
        names_chunk = names_chunk[names_chunk['lastname3']==lastname3[0]]
        filter_name = 'lastname4'
    if lastname4:
        names_chunk = names_chunk[names_chunk['lastname4']==lastname4[0]]
        filter_name = 'lastname5'
    
    return names_chunk[filter_name].value_counts(ascending=False).head(size).items()
        
top_target = {}
root_names = find_name_leaf()
for root_name in tqdm(root_names):
    if root_name not in top_target:
        top_target[root_name] = {}

    root_names2 = find_name_leaf(lastname1=root_name)
    for root_name2 in root_names2:
        if root_name2 not in top_target[root_name]:
            top_target[root_name][root_name2] = {}

        root_names3 = find_name_leaf(lastname1=root_name,lastname2=root_name2)
        for root_name3 in root_names3:
            if root_name3 not in top_target[root_name][root_name2]:
                top_target[root_name][root_name2][root_name3] = {}

            root_names4 = find_name_leaf(lastname1=root_name,lastname2=root_name2,lastname3=root_name3)
            for root_name4 in root_names4:
                root_names5 = find_name_leaf(lastname1=root_name,lastname2=root_name2,lastname3=root_name3,lastname4=root_name4)
                root_names5 = [x for x in root_names5]
                if root_name4 not in top_target[root_name][root_name2][root_name3]:
                    top_target[root_name][root_name2][root_name3][root_name4] = root_names5


top_target



5it [09:23, 112.72s/it]


{('HIDAYAT',
  172827): {('RAHMAT', 13974): {('ASEP', 270): {},
   ('ADE', 127): {},
   ('DEDE', 123): {},
   ('AGUS', 82): {},
   ('UJANG', 75): {}}, ('TAUFIK', 8481): {('MUHAMAD', 138): {},
   ('ASEP', 91): {},
   ('MUHAMMAD', 83): {('ASEP', 1): []},
   ('AHMAD', 62): {('ASEP', 1): []},
   ('AGUS', 40): {('MOHAMAD', 1): []}}, ('SARIP', 5225): {('ASEP', 159): {},
   ('YAYAT', 69): {('U', 1): [], ('RD', 1): []},
   ('IIP', 63): {('ASEP', 1): []},
   ('DEDE', 62): {},
   ('AHMAD', 54): {}}, ('YAYAT', 4898): {('ASEP', 27): {},
   ('ADE', 17): {('M', 1): []},
   ('UJANG', 10): {},
   ('H', 10): {},
   ('DEDEN', 6): {}}, ('AHMAD', 4353): {('ASEP', 91): {},
   ('YAYAT', 52): {},
   ('DEDE', 42): {},
   ('ADE', 33): {},
   ('UJANG', 31): {}}},
 ('NURHAYATI',
  142130): {('SITI', 7426): {('AI', 109): {('HJ', 1): []},
   ('NENG', 56): {},
   ('IMAS', 27): {},
   ('EUIS', 26): {},
   ('YATI', 22): {}}, ('NUNUNG', 5307): {('HJ', 17): {('DRA', 1): []},
   ('SITI', 5): {},
   ('DEDE', 5): {},
   (

In [91]:
top_target.keys()

dict_keys([('HIDAYAT', 172827), ('NURHAYATI', 142130), ('SETIAWAN', 136935), ('SARI', 128651), ('GUNAWAN', 96324)])

In [63]:
top_target[('HIDAYAT', 172827)].keys()

dict_keys([('RAHMAT', 13974), ('TAUFIK', 8481), ('SARIP', 5225), ('YAYAT', 4898), ('AHMAD', 4353)])

In [61]:
top_target[('HIDAYAT', 107315)][('TAUFIK', 5512)]

KeyError: ('HIDAYAT', 107315)

In [92]:
def formating_name(list_tt):
    full_name = ''
    for tt in list_tt:
        full_name = full_name + tt[0] + '\n'
    
    full_name += '({})'.format(list_tt[0][1])
    return full_name

source_target = []
counter = 0
for tt in top_target:
    source_target.append(dict(
        target = None,
        # source = '(1)'+tt[0],
        source = formating_name([tt]),
        value = tt[1],
        y=counter/len(top_target)
    ))
    counter+=1.0

def clean_name(source_target,param = '(1)'):
    clean_names = []
    for src in source_target:
        src_name = src['source']
        if param not in src_name:
            continue
        else:
            src_name = src_name.replace(param,'')
            clean_names.append(src_name)
    return clean_names

counter = 0
for tt1 in top_target:
    for tt in top_target[tt1]:
        source_target.append(dict(
        # source = '(2)'+tt[0],
        # target = '(1)'+tt1[0],
        source = formating_name([tt,tt1]),
        target = formating_name([tt1]),

        value = tt[1],
        y = counter/len(top_target[tt1])
        ))
        counter+=1.0

counter = 0
for tt1 in top_target:
    for tt2 in top_target[tt1]:
        for tt in top_target[tt1][tt2]:
            source_target.append(dict(
            # source = '(3)'+tt[0],
            # target = '(2)'+tt2[0],
            source = formating_name([tt,tt2,tt1]),
            target = formating_name([tt2,tt1]),

            value = tt[1],
            y = counter/len(top_target[tt1])
            ))
            counter+=1.0
                
            
counter = 0
for tt1 in top_target:
    for tt2 in top_target[tt1]:
        for tt3 in top_target[tt1][tt2]:
            for tt in top_target[tt1][tt2][tt3]:
                source_target.append(dict(
                # source = '(4)'+tt[0],
                # target = '(3)'+tt3[0],        
                source = formating_name([tt,tt3,tt2,tt1]),
                target = formating_name([tt3,tt2,tt1]),

                value = tt[1],
                y = counter/len(top_target[tt1])
                ))
                counter+=1.0
            

# for tt1 in top_target:
#     for tt2 in top_target[tt1]:
#         for tt3 in top_target[tt1][tt2]:
#             for key,tt4 in zip(clean_name(source_target,'(4)'),top_target[tt1][tt2][tt3]):
#                 for tt in top_target[tt1][tt2][tt3][tt4]:
#                     source_target.append(dict(
#                     source = '(5)'+tt[0],
#                     target = '(4)'+key,
#                     value = tt[1]
#                 ))

source_target_df = pd.DataFrame(source_target)
source_target_df.head(2)


,target,source,value,y
0,None,HIDAYAT\n(172827),172827,0.0
1,None,NURHAYATI\n(142130),142130,0.2


## First name

In [106]:
from tqdm import tqdm
def find_firstname_leaf(firstname1=None,firstname2=None,firstname3=None,firstname4=None,size=5):
    names_chunk = pd.DataFrame()
    filter_name = '1stname'
    if firstname1 == None:
        names_chunk = names.copy()
    if firstname1:
        names_chunk = names[names['1stname']==firstname1[0]]
        filter_name = '2ndname'
    if firstname2:
        names_chunk = names_chunk[names_chunk['2ndname']==firstname2[0]]
        filter_name = '3rdname'
    if firstname3:
        names_chunk = names_chunk[names_chunk['3rdname']==firstname3[0]]
        filter_name = '4thname'
    if firstname4:
        names_chunk = names_chunk[names_chunk['4thname']==firstname4[0]]
        filter_name = '5thname'
    
    return names_chunk[filter_name].value_counts(ascending=False).head(size).items()
        
top_first_target = {}
root_names = find_firstname_leaf()
for root_name in tqdm(root_names):
    if root_name not in top_first_target:
        top_first_target[root_name] = {}

    root_names2 = find_firstname_leaf(firstname1=root_name)
    for root_name2 in root_names2:
        if root_name2 not in top_first_target[root_name]:
            top_first_target[root_name][root_name2] = {}

        root_names3 = find_firstname_leaf(firstname1=root_name,firstname2=root_name2)
        for root_name3 in root_names3:
            if root_name3 not in top_first_target[root_name][root_name2]:
                top_first_target[root_name][root_name2][root_name3] = {}

            root_names4 = find_firstname_leaf(firstname1=root_name,firstname2=root_name2,firstname3=root_name3)
            for root_name4 in root_names4:
                root_names5 = find_firstname_leaf(firstname1=root_name,firstname2=root_name2,firstname3=root_name3,firstname4=root_name4)
                root_names5 = [x for x in root_names5]
                if root_name4 not in top_first_target[root_name][root_name2][root_name3]:
                    top_first_target[root_name][root_name2][root_name3][root_name4] = root_names5


top_first_target



5it [11:59, 143.82s/it]


{('SITI',
  635073): {('AMINAH',
   19741): {('BT', 169): {('AMAD', 3): [],
    ('AMIN', 3): [],
    ('ADANG', 3): [],
    ('SUEB', 2): [],
    ('ENGGIN', 2): []}, ('BINTI', 41): {('MISAR', 2): [],
    ('TJARLI', 2): [],
    ('AMIT', 2): [],
    ('KASAM', 2): [],
    ('SANUSI', 2): []}, ('HJ', 21): {}, ('LUBIS', 7): {}, ('NASUTION',
    7): {}}, ('AISAH',
   19570): {('BT', 159): {('H', 4): [('MAHIRIN', 2), ('SUNARYA', 2)],
    ('SAIM', 3): [],
    ('DARMA', 3): [],
    ('MURHAN', 2): [],
    ('AMIN', 2): [('RAIS', 2)]}, ('BINTI', 53): {('NAJMUDIN', 2): [],
    ('DEDIH', 2): [],
    ('SANAN', 2): [],
    ('SOBUR', 2): [],
    ('ENDA', 2): []}, ('HJ', 15): {}, ('BIN', 13): {('ACIN', 2): [],
    ('HAJI', 2): [],
    ('KANCRA', 1): [],
    ('IJUDIN', 1): [],
    ('ASAN', 1): []}, ('NURHASANAH', 10): {}}, ('AISYAH',
   15422): {('BT', 76): {('YAHMAN', 2): [],
    ('RASPIN', 2): [],
    ('ZAENAL', 2): [('ABIDIN', 2)],
    ('NARSUM', 2): [],
    ('DUL', 2): [('SALAM', 2)]}, ('BINTI', 24): {(

In [107]:
top_first_target.keys()

dict_keys([('SITI', 635073), ('ASEP', 288489), ('ADE', 256033), ('DEDE', 246211), ('SRI', 217163)])

In [115]:
def formating_firstname(list_tt):
    full_name = ''
    for tt in list_tt:
        full_name = full_name + tt[0] + '\n'
    
    full_name += '({})'.format(list_tt[-1][1])
    return full_name

source_first_target = []
counter = 0
for tt in top_first_target:
    source_first_target.append(dict(
        target = None,
        # source = '(1)'+tt[0],
        source = formating_firstname([tt]),
        value = tt[1],
    ))
    counter+=1.0

def clean_name(source_first_target,param = '(1)'):
    clean_names = []
    for src in source_first_target:
        src_name = src['source']
        if param not in src_name:
            continue
        else:
            src_name = src_name.replace(param,'')
            clean_names.append(src_name)
    return clean_names

counter = 0
for tt1 in top_first_target:
    for tt in top_first_target[tt1]:
        source_first_target.append(dict(
        # source = '(2)'+tt[0],
        # target = '(1)'+tt1[0],
        source = formating_firstname([tt1,tt]),
        target = formating_firstname([tt1]),

        value = tt[1]
        ))
        counter+=1.0

counter = 0
for tt1 in top_first_target:
    for tt2 in top_first_target[tt1]:
        for tt in top_first_target[tt1][tt2]:
            source_first_target.append(dict(
            # source = '(3)'+tt[0],
            # target = '(2)'+tt2[0],
            source = formating_firstname([tt1,tt2,tt]),
            target = formating_firstname([tt1,tt2]),

            value = tt[1]
            ))
            counter+=1.0
                
            
counter = 0
for tt1 in top_first_target:
    for tt2 in top_first_target[tt1]:
        for tt3 in top_first_target[tt1][tt2]:
            for tt in top_first_target[tt1][tt2][tt3]:
                source_first_target.append(dict(
                # source = '(4)'+tt[0],
                # target = '(3)'+tt3[0],        
                source = formating_firstname([tt1,tt2,tt3,tt]),
                target = formating_firstname([tt1,tt2,tt3]),

                value = tt[1]
                ))
                counter+=1.0
            

# for tt1 in top_first_target:
#     for tt2 in top_first_target[tt1]:
#         for tt3 in top_first_target[tt1][tt2]:
#             for key,tt4 in zip(clean_name(source_first_target,'(4)'),top_first_target[tt1][tt2][tt3]):
#                 for tt in top_first_target[tt1][tt2][tt3][tt4]:
#                     source_first_target.append(dict(
#                     source = '(5)'+tt[0],
#                     target = '(4)'+key,
#                     value = tt[1]
#                 ))

source_first_target_df = pd.DataFrame(source_first_target)
source_first_target_df.head(2)


,target,source,value
0,None,SITI\n(635073),635073
1,None,ASEP\n(288489),288489


### sankey

In [169]:
nodes = []
def find_position(label):
    if '(1)' in label:
        return 1
    if '(2)' in label:
        return 0.75
    if '(3)' in label:
        return 0.5
    if '(4)' in label:
        return 0.25
    if '(5)' in label:
        return 0
    
for st in source_target:
    if st['source']:
        nodes.append({'label':st['source'],'x':find_position(st['source']),'y':st['y']})
    if st['target']:
        nodes.append({'label':st['target'],'x':find_position(st['source']),'y':st['y']})

def find_nodes(label):
    if not label:
        return ''
    for n in nodes:
        if n['label'] == label:
            return nodes.index(n)
    

links = []
for st in source_target:
    links.append({
        'source':find_nodes(st['source']),
        'target':find_nodes(st['target']),
        'value':st['value']
    })

In [170]:
[1,2,'er',4,{3:3}].index({3:3})

4

In [171]:
import plotly.graph_objects as go

# Define the nodes and links
# nodes = [
#     {'label': 'Group A'},
#     {'label': 'Group B'},
#     {'label': 'Group C'},
# ]

# links = [
#     {'source': 0, 'target': 1, 'value': 50},   # Flow from Group A to Group B with a value of 50
#     {'source': 0, 'target': 2, 'value': 30},   # Flow from Group A to Group C with a value of 30
#     {'source': 1, 'target': 2, 'value': 20},   # Flow from Group B to Group C with a value of 20
# ]

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color='black', width=0.5),
        label=[node['label'] for node in nodes],
        # x=[node['x'] for node in nodes], 
        # y=[node['y'] for node in nodes], 
    ),
    link=dict(
        source=[link['source'] for link in links],
        target=[link['target'] for link in links],
        value=[link['value'] for link in links],
    ),
    # arrangement="fixed"  # Set the arrangement to "fixed"
)])


# Customize the layout
fig.update_layout(
    title='Flow of People between Groups',
    font=dict(size=12, color='black'),
    # Set the figure size
    width=800,
    height=2000
)

# Display the figure
fig.show()


### ete

In [173]:
!pip install ete3

     ---------------------------------------- 4.8/4.8 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for ete3: started
  Running setup.py install for ete3: finished with status 'done'


  DEPRECATION: ete3 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [179]:
import ete3
import importlib
importlib.reload(ete3)

<module 'ete3' from 'c:\\Users\\yosia\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\ete3\\__init__.py'>

In [181]:
!pip uninstall PyQt5
!pip install PyQt4

^C


ERROR: Could not find a version that satisfies the requirement PyQt4 (from versions: none)
ERROR: No matching distribution found for PyQt4

[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [184]:
from ete3 import Tree, TreeStyle
t = Tree()
t.populate(30)
ts = TreeStyle()
ts.show_leaf_name = True
ts.mode = "c"
ts.arc_start = -180 # 0 degrees = 3 o'clock
ts.arc_span = 180
# t.show(tree_style=ts)
t.render("mytree.svg",)
# t.write('mytree.svg')

'(((aaaaaaaaac:1,(aaaaaaaaad:1,(aaaaaaaaae:1,aaaaaaaaaf:1)1:1)1:1)1:1,((aaaaaaaaag:1,aaaaaaaaah:1)1:1,(aaaaaaaaai:1,aaaaaaaaaj:1)1:1)1:1)1:1,(((aaaaaaaaak:1,aaaaaaaaal:1)1:1,(aaaaaaaaam:1,(aaaaaaaaan:1,aaaaaaaaao:1)1:1)1:1)1:1,(((aaaaaaaaap:1,aaaaaaaaaq:1)1:1,(aaaaaaaaar:1,aaaaaaaaas:1)1:1)1:1,((aaaaaaaaat:1,(aaaaaaaaau:1,(aaaaaaaaav:1,(aaaaaaaaaw:1,(aaaaaaaaax:1,(aaaaaaaaay:1,aaaaaaaaaz:1)1:1)1:1)1:1)1:1)1:1)1:1,((aaaaaaaabb:1,(aaaaaaaabc:1,(aaaaaaaabd:1,aaaaaaaabe:1)1:1)1:1)1:1,(aaaaaaaaaa:1,aaaaaaaaab:1)1:1)1:1)1:1)1:1)1:1);'

### pyvis

In [192]:
from pyvis.network import Network
import networkx as nx

g = Network(notebook=True)
import networkx as nx

# Create a sample DataFrame
df = pd.DataFrame({
    'source': ['Node 1', 'Node 1', 'Node 2', 'Node 3'],
    'target': ['Node 2', 'Node 3', 'Node 3', 'Node 4'],
    'value': [20, 30, 40, 50]
})

# Create an empty graph
G = nx.from_pandas_edgelist(df, 'source', 'target')

# Map node sizes based on 'value' column
node_sizes = [50 * value for value in df['value']]

# Draw the graph
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=node_sizes)


nxg = nx.complete_graph(4)
g.from_nx(G)
g.save_graph('graph.html')

# nx_graph = nx.cycle_graph(10)
# nx_graph.nodes[1]['title'] = 'Number 1'
# nx_graph.nodes[1]['group'] = 1
# nx_graph.nodes[3]['title'] = 'I belong to a different group!'
# nx_graph.nodes[3]['group'] = 10
# nx_graph.add_node(20, size=20, title='couple', group=2)
# nx_graph.add_node(21, size=15, title='couple', group=2)
# nx_graph.add_edge(20, 21, weight=5)
# nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)
# nt = Network('500px', '500px',notebook=True,cdn_resources='remote')
# # populates the nodes and edges data structures
# nt.from_nx(nx_graph)
# nt.show('nx.html')

ModuleNotFoundError: No module named 'matplotlib'

In [20]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

data = {
    'source': ['A', 'B', 'C'],
    'target': ['B', 'C', 'A'],
    'value': [10, 15, 5]
}

df = pd.DataFrame(data)

G = nx.Graph()

for row in df.itertuples(index=False):
    G.add_edge(row.source, row.target, value=row.value)

nt = Network(height='500px', width='800px')
nt.from_nx(G)
# nt.show('network.html')
nt.save_graph('graph.html')


In [78]:
source_target_df.head(20)

,target,source,value,y
0,None,HIDAYAT\n(172827),172827,0.0
1,HIDAYAT\n(172827),RAHMAT\nHIDAYAT\n(13974),13974,0.0
2,HIDAYAT\n(172827),TAUFIK\nHIDAYAT\n(8481),8481,0.2
3,HIDAYAT\n(172827),SARIP\nHIDAYAT\n(5225),5225,0.4
4,HIDAYAT\n(172827),YAYAT\nHIDAYAT\n(4898),4898,0.6
5,HIDAYAT\n(172827),AHMAD\nHIDAYAT\n(4353),4353,0.8
6,ASEP\nRAHMAT\n(270),ASEP\nRAHMAT\nHIDAYAT\n(270),270,0.0
7,ADE\nRAHMAT\n(127),ADE\nRAHMAT\nHIDAYAT\n(127),127,0.2
8,DEDE\nRAHMAT\n(123),DEDE\nRAHMAT\nHIDAYAT\n(123),123,0.4
9,AGUS\nRAHMAT\n(82),AGUS\nRAHMAT\nHIDAYAT\n(82),82,0.6


In [93]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

# data = {
#     'source': ['A', 'B', 'C'],
#     'target': ['B', 'C', 'A'],
#     'value': [20, 15, 5],
#     'node_size': [10, 30, 15]
# }

# df = pd.DataFrame(data)

graph_df = source_target_df.copy()

G = nx.Graph()

for row in graph_df.itertuples(index=False):
    if not row.target:
        continue
    # G.add_edge(row.source, row.target, value=row.value/100)
    G.add_edge(row.source, row.target)
    # G.add_edge(1, 3, weight=7, capacity=15, length=342.7)

# nt = Network(height='500px', width='800px',directed=True)
nt = Network(height='500px', width='800px')

# Add nodes to the network with their respective sizes
for node in G.nodes:
    if node:
        node_size = graph_df.loc[graph_df['source'] == node, 'value'].values[0]
        node_size = int(node_size)
        if node_size > 100000:
            node_size = 40
        elif node_size > 10000:
            node_size = 30
        elif node_size > 1000:
            node_size = 20
        else:
            node_size = 10
        # nt.add_node(node, size=node_size)
        nt.add_node(node, size=node_size)
        # nt.add_node(node)

# Add edges to the network
# for edge in G.edges:
#     nt.add_edge(edge[0], edge[1])
nt.from_nx(G)
nt.save_graph('graph.html')

In [84]:
graph_df

,target,source,value,y
0,None,HIDAYAT\n(172827),172827,0.0
1,HIDAYAT\n(172827),RAHMAT\nHIDAYAT\n(13974),13974,0.0
2,HIDAYAT\n(172827),TAUFIK\nHIDAYAT\n(8481),8481,0.2
3,HIDAYAT\n(172827),SARIP\nHIDAYAT\n(5225),5225,0.4


In [117]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

# data = {
#     'source': ['A', 'B', 'C'],
#     'target': ['B', 'C', 'A'],
#     'value': [20, 15, 5],
#     'node_size': [10, 30, 15]
# }

# df = pd.DataFrame(data)

graph_df = source_first_target_df.copy()

G = nx.Graph()

for row in graph_df.itertuples(index=False):
    if not row.target:
        continue
    # G.add_edge(row.source, row.target, value=row.value/100)
    G.add_edge(row.source, row.target)
    # G.add_edge(1, 3, weight=7, capacity=15, length=342.7)

# nt = Network(height='500px', width='800px',directed=True)
nt = Network(height='500px', width='800px')

# Add nodes to the network with their respective sizes
for node in G.nodes:
    if node:
        node_size = graph_df.loc[graph_df['source'] == node, 'value'].values[0]
        node_size = int(node_size)
        if node_size > 500000:
            node_size = 70
        elif node_size > 250000:
            node_size = 50
        elif node_size > 200000:
            node_size = 30
        elif node_size > 15000:
            node_size = 20
        elif node_size > 7000:
            node_size = 10
        else:
            node_size = 5
        # nt.add_node(node, size=node_size)
        nt.add_node(node, size=node_size)
        # nt.add_node(node)

# Add edges to the network
# for edge in G.edges:
#     nt.add_edge(edge[0], edge[1])
nt.from_nx(G)
nt.save_graph('firstname_graph.html')

In [116]:
source_first_target_df.sort_values('value',ascending=False).head(30)

,target,source,value
0,None,SITI\n(635073),635073
1,None,ASEP\n(288489),288489
2,None,ADE\n(256033),256033
3,None,DEDE\n(246211),246211
4,None,SRI\n(217163),217163
5,SITI\n(635073),SITI\nAMINAH\n(19741),19741
6,SITI\n(635073),SITI\nAISAH\n(19570),19570
7,SITI\n(635073),SITI\nAISYAH\n(15422),15422
8,SITI\n(635073),SITI\nNURJANAH\n(13962),13962
25,SRI\n(217163),SRI\nRAHAYU\n(13150),13150


In [103]:
names[names['1stname']=='SITI']

,nama,unique,1stname,2ndname,3rdname,4thname,5thname,lastname1,lastname2,lastname3,lastname4,lastname5
38,SITI AISAH,NaN,SITI,AISAH,NaN,NaN,NaN,AISAH,SITI,NaN,NaN,NaN
75,SITI HAJAR,NaN,SITI,HAJAR,NaN,NaN,NaN,HAJAR,SITI,NaN,NaN,NaN
85,SITI ZENAB,NaN,SITI,ZENAB,NaN,NaN,NaN,ZENAB,SITI,NaN,NaN,NaN
110,SITI NURJANAH,NaN,SITI,NURJANAH,NaN,NaN,NaN,NURJANAH,SITI,NaN,NaN,NaN
116,SITI NURJANAH,NaN,SITI,NURJANAH,NaN,NaN,NaN,NURJANAH,SITI,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
32049337,SITI AMINAH,NaN,SITI,AMINAH,NaN,NaN,NaN,AMINAH,SITI,NaN,NaN,NaN
32049532,SITI NURLAELAH,NaN,SITI,NURLAELAH,NaN,NaN,NaN,NURLAELAH,SITI,NaN,NaN,NaN
32049536,SITI NURMILAH,NaN,SITI,NURMILAH,NaN,NaN,NaN,NURMILAH,SITI,NaN,NaN,NaN
32049591,SITI HERLINA,NaN,SITI,HERLINA,NaN,NaN,NaN,HERLINA,SITI,NaN,NaN,NaN


In [104]:
names[names['lastname1']=='PUTRA']

,nama,unique,1stname,2ndname,3rdname,4thname,5thname,lastname1,lastname2,lastname3,lastname4,lastname5
2364,ASEP DIANA PUTRA,NaN,ASEP,DIANA,PUTRA,NaN,NaN,PUTRA,DIANA,ASEP,NaN,NaN
4884,JHONI EKA PUTRA,NaN,JHONI,EKA,PUTRA,NaN,NaN,PUTRA,EKA,JHONI,NaN,NaN
6196,M. GILANG KUSUMA PRATAMA PUTRA,NaN,GILANG,KUSUMA,PRATAMA,PUTRA,NaN,PUTRA,PRATAMA,KUSUMA,GILANG,NaN
6207,AWAL BAKTI SATRIA PUTRA,NaN,AWAL,BAKTI,SATRIA,PUTRA,NaN,PUTRA,SATRIA,BAKTI,AWAL,NaN
6719,ANGGA PRATAMA PUTRA,NaN,ANGGA,PRATAMA,PUTRA,NaN,NaN,PUTRA,PRATAMA,ANGGA,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
32042857,ALFA SABILLA SUGANDA PUTRA,NaN,ALFA,SABILLA,SUGANDA,PUTRA,NaN,PUTRA,SUGANDA,SABILLA,ALFA,NaN
32044456,EKI IRAWAN PUTRA,NaN,EKI,IRAWAN,PUTRA,NaN,NaN,PUTRA,IRAWAN,EKI,NaN,NaN
32044863,ADE PANCA PUTRA,NaN,ADE,PANCA,PUTRA,NaN,NaN,PUTRA,PANCA,ADE,NaN,NaN
32045603,CECEP SUHENA PUTRA,NaN,CECEP,SUHENA,PUTRA,NaN,NaN,PUTRA,SUHENA,CECEP,NaN,NaN


In [121]:
fav_first_name = names.value_counts('1stname').to_frame().reset_index()
fav_first_name

,1stname,count
0,SITI,635073
1,ASEP,288489
2,ADE,256033
3,DEDE,246211
4,SRI,217163
...,...,...
743037,FUNDWI,1
743038,NUSRULLOH,1
743039,FUNFUN,1
743040,NUSROWATI,1


In [122]:
import plotly.express as px
fig = px.bar(fav_first_name.head(10), x='1stname', y='count',text_auto=True, title='Nama Depan Pasaran')
fig.show()